1. 학습데이터, 테스트데이터 만들기
먼저 모델을 만들고 나서 학습을 시킬 데이터를 만들기 위해 네이버에서 몇몇의 기업을 선정하고

그 기업에 대한 기사의 제목을 크롤링하였습니다.

 

크롤링 후에 학습시 필요한 긍정, 부정, 중립을 나타내는 label이 있었어야했는데

손으로 일일이 부정, 긍정, 중립 이 세 가지로 label을 붙이려고 하다보니

눈도 아프고 비슷한 내용임에도 불구하고 앞에서는 긍정으로 했다가 뒤에서는 중립으로 표기하는 등의 문제가 있었습니다.

 

이를 컴퓨터가 긍정적인 단어, 부정적인 단어가 포함되어있는지 여부를 확인하여

자동으로 라벨을 붙여주면 편할 것 같아 그렇게 만들어 보았습니다.

 

먼저, 긍정적인 단어, 부정적인 단어가 포함된 txt파일을 각각 만들어주었습니다.

negative_words_self.txt
0.00MB
positive_words_self.txt
0.00MB
뉴스 기사를 보며 만든 긍정적인 단어, 부정적인 단어 모음입니다.

단어는 생각나는대로 계속 추가하고자합니다.

 

코드에서는 이 단어들을 파일에서 positive, negative라는 list로 받아와서

두 개의 list를 합쳐 posneg라는 list를 만들고 크롤링해오는 단어에서 posneg안에 있는 단어가 포함되어있으면 긍정, 부정 라벨을 붙여주고

포함되어있지 않으면 그냥 중립인 0의 상태로 그대로 두도록 만들어 보았습니다.

 

파일에서 단어를 불러와 posneg리스트를 만드는 코드

In [1]:
import codecs

positive = []
negative = []
posneg = []

pos = codecs.open("./positive_words_self.txt", 'rb', encoding='UTF-8')

while True:
  line = pos.readline()
  line = line.replace('\n', '')
  positive.append(line)
  posneg.append(line)
  
  if not line: break
pos.close()



neg = codecs.open("./negative_words_self.txt", 'rb', encoding='UTF-8')

while True:
  line = neg.readline()
  line = line.replace('\n', '')
  negative.append(line)
  posneg.append(line)
  
  if not line: break
neg.close()

크롤링한 기사 제목과 기사 제목과 posneg를 활용하여 만든 긍정(1), 부정(-1), 중립(0)라벨 정보를 가지는 dataframe을 만드는 함수

(예시 : 네이버에서 버거킹으로 검색하여 나온 기사 4,000개 제목과 각각 제목의 긍정, 부정, 중립 라벨 생성)

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


label = [0] * 4000

my_title_dic = {"title":[], "label":label}

j = 0


for i in range(400):
  num = i * 10 + 1
  # bhc
# url = "https://search.naver.com/search.naver?&where=news&query=bhc&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=23&start=" + str(num)
  # 아오리라멘
# url2 = "https://search.naver.com/search.naver?&where=news&query=%EC%95%84%EC%98%A4%EB%A6%AC%EB%9D%BC%EB%A9%98&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=34&start=" + str(num)
  
  # 버거킹
  url3 = "https://search.naver.com/search.naver?&where=news&query=%EB%B2%84%EA%B1%B0%ED%82%B9&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=23&start=" + str(num)
  
  req = requests.get(url3)
  
  soup = BeautifulSoup(req.text, 'lxml')
  
  titles = soup.select("a._sp_each_title")
  
  for title in titles:
    
    title_data = title.text
    title_data = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》]', '', title_data)
    my_title_dic['title'].append(title_data)
    
    for i in range(len(posneg)):
      posflag = False
      negflag = False
      if i < (len(positive)-1):
# print(title_data.find(posneg[i]))
        if title_data.find(posneg[i]) != -1:
          posflag = True
          print(i, "positive?","테스트 : ",title_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i, title_data)
          break
      if i > (len(positive)-2):
        if title_data.find(posneg[i]) != -1:
          negflag = True
          print(i, "negative?","테스트 : ",title_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i, title_data)
          break
    if posflag == True:
      label[j] = 1
# print("positive", j)
    elif negflag == True:
      label[j] = -1
# print("negative", j)
    elif negflag == False and posflag == False:
      label[j] = 0
# print("objective", j)
    j = j + 1
my_title_dic['label'] = label
my_title_df = pd.DataFrame(my_title_dic)

6 positive? 테스트 :  24 비교단어 :  수상 인덱스 :  6 사딸라 CF 상받았다버거킹 에피어워드 은상 수상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 주문 오래 걸린다 美 버거킹서 직원 총으로 쏴 살해
6 positive? 테스트 :  27 비교단어 :  수상 인덱스 :  6 버거킹 올데이킹 사딸라’ 캠페인 에피어워드 은상 수상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼 11 프로모션 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 2초 안에 햄버거 만들어 와 美 버거킹 점원에 탕
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 글로벌Biz 24 버거킹 웬디스 스윗그린 등 미 유명 패스트푸드 포장지에서
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 美 버거킹 등 햄버거 포장재서 과불화화합물 검출국내 문제없어  
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 알바생이 손님에게 풀스윙’ 뺨 맞은 이유 영상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 뉴스피처 소가 뀌는 방귀 줄일게요햄버거 회사가 왜 이런 광고를
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 영상 왜 너겟 적게 줘버거킹 직원 뺨 때리고 욕 퍼부은 남성
8 positive? 테스트 :  4 비교단어 :  인기 인덱스 :  8 버거킹 인기 버거 2개 5000원 프로모션 진행
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 애니멀고·스타벅스·버거킹·편의점 물품 등 50 할인 핫딜 GOM2 곰투마트
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 2020년 끝내자 美 버거킹이 7월에 크리스마스 행사 하는 이유
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 유통가 투데이 일화

46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 사딸라로 빵 터진 버거킹 400호점 열었다
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼주니어·불고기와퍼주니어 단품 2000원에 판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼 및 불고기와퍼 단품 40 할인 프로모션 진행
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 소 방귀를 줄여라 버거킹이 내놓은 기발한 메탄 감소 캠페인
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 프리미엄 와퍼 3종 각 3900원에 판매 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 오늘 뭐 먹지 폴 바셋·교촌치킨·버거킹 외
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 소 방귀 덜 뀌게 이것 먹인다메탄 배출 감소
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 햄버거도 구독 서비스 버거킹 월 4700원에 버거 4개
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼주니어 11 프로모션 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 중국서 기한 넘은 재료 썼다가 혼쭐매장 폐쇄
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 햄버거도 구독 서비스 버거킹 월 4700원에 버거 4개
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼주니어 11 프로모션 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 프리미엄 햄버거 전략 통했다400호점
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 중국서 기한 넘은 재료 썼다가 혼쭐매장 폐쇄
46 negative? 테스트 :  0 비교단어 :   인덱스 

46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼주니어 11 프로모션 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 신규 메뉴 더블롱킹 세트 4900원에 한정 판매
15 positive? 테스트 :  30 비교단어 :  출시 인덱스 :  15 대게 버거 되게 궁금하네버거킹 바삭·촉촉 붉은대게와퍼 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹은 왜 곰팡이 핀 와퍼 광고를 했을까
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 코카콜라 둥지냉면 버거킹 연말 줄줄이 가격 인상
15 positive? 테스트 :  31 비교단어 :  출시 인덱스 :  15 버거킹으로 떠나는 싱가포르 미식여행버거킹 칠리크랩 버거 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 이달 12일까지 와퍼주니어 2천원 프로모션
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 불고기버거 단품 할인 프로모션 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 4딸라보다 싸다버거킹 일주일간 와퍼가 3500원
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 프리미엄 와퍼 3종 할인행사 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 4딸라보다 싸다버거킹 일주일간 와퍼가 3500원
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 신규 메뉴 더블롱킹 세트’ 4900원에 한정 판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 프리미엄 와퍼 3종 할인행사 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼 등 최대 300원 인상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와

15 positive? 테스트 :  18 비교단어 :  출시 인덱스 :  15 꼭 먹어봐야 버거킹 붉은대게와퍼 출시 화제
4 positive? 테스트 :  17 비교단어 :  돌파 인덱스 :  4 버거킹 36년 만에 400호점 돌파포천축석휴게소FS점 오픈
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 주지훈 버거킹 새 모델 발탁킹과 킹의 만남
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 단 7일간 와퍼주니어 1900원 판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 오늘20일부터 대표 메뉴 와퍼 40 할인 행사 진행
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼 불향 살린 반려견 간식 독퍼’ 무료 증정
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼 등 27개 메뉴 25 가격 인상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 커피구독 서비스 오퀴즈 진행
15 positive? 테스트 :  12 비교단어 :  출시 인덱스 :  15 버거킹 붉은대게 와퍼 출시 4주만에 100만개 판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 신규 메뉴 더블롱킹 세트 4900원에 한정 판매
15 positive? 테스트 :  12 비교단어 :  출시 인덱스 :  15 버거킹 붉은대게 와퍼 출시 4주만에 100만개 판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 신규 메뉴 더블롱킹 세트 4900원에 한정 판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 독퍼 이벤트반려견 모시는 집사들 모여라종합
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 中 2020년 315 소비자 고발 프로그램 이번엔 누가
46 negative? 테스트 :  0 비

17 positive? 테스트 :  28 비교단어 :  오픈 인덱스 :  17 버거킹 국내 매장 400호점 포천축석휴게소FS점’ 오픈
15 positive? 테스트 :  20 비교단어 :  출시 인덱스 :  15 버거킹 치킨윙 사이드 신메뉴 바삭킹 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 23일까지 프리미엄와퍼 3종 3900원에 판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 OK캐쉬백 버거킹 커피구독’ 초성 퀴즈 출제 정답은
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 27개 메뉴 가격 인상와퍼 200원 오른다
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 묻고 더블로 가’ 김응수 모델 발탁
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹X씨그램 제로 칼로리 제로톡톡 3총 줄시
8 positive? 테스트 :  10 비교단어 :  인기 인덱스 :  8 버거킹 일주일 간 인기 와퍼 3종 프로모션 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 19일까지 와퍼주니어 버거 3종 할인 판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼주니어 할인행사
15 positive? 테스트 :  21 비교단어 :  출시 인덱스 :  15 버거킹 신규 메뉴 스윗마요 킹치킨버거 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 대표 메뉴 와퍼 40 할인 프로모션 실시
8 positive? 테스트 :  17 비교단어 :  인기 인덱스 :  8 버거킹 킹스초이스 프로모션 실시인기 와퍼 2개 7000원
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 7일간 프리미엄 와퍼 3종’ 4200원 프로모션 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 : 

...

46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 콰트로레이스 프로모션
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹도 백기’ 어린이 메뉴서 탄산음료 제외 앞서 맥도널도 웬디스도 
4 positive? 테스트 :  21 비교단어 :  돌파 인덱스 :  4 버거킹 콰트로치즈와퍼 100만개 판매 돌파
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 뉴스투데이 E 버거킹 사딸라” 김영철 카톡 이모티콘 증정
6 positive? 테스트 :  29 비교단어 :  수상 인덱스 :  6 버거킹 통새우와퍼 광고 서울영상광고제 2016 동상 수상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 매장가→배달비’ 이름만 바꿔 또 가격인상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 강렬한 레드번이 특징인 앵그리몬스터X 10월 한정판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 9일까지 불고기버거 1900원에 판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 치폴레와퍼 75만개 판매 달성
8 positive? 테스트 :  18 비교단어 :  인기 인덱스 :  8 곽철용 김응수 버거킹 CF 티저 인기몰이젊은 친구 취향 저격 성공
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 몬스터 시리즈 정식 메뉴 확정
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 지하철 옥외 광고 게재
8 positive? 테스트 :  4 비교단어 :  인기 인덱스 :  8 버거킹 인기 사이드 메뉴가 천원 21 치즈스틱선데 할인행사
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 콰트로치즈와퍼‧치즈와퍼 단품 3900원에 판매40 할인가
15 positive? 테스트 :  15 비교단어 :

46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹·롯데리아·맥도날드 대표 햄버거 프랜차이즈 8월 이벤트 열전
15 positive? 테스트 :  13 비교단어 :  출시 인덱스 :  15 버거킹 해쉬브라운와퍼’ 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 20대들이 가장 선호하는 패스트푸드 브랜드는 버거킹
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 설날 와퍼팩 프로모션 와퍼팩 9900원에 판매
15 positive? 테스트 :  21 비교단어 :  출시 인덱스 :  15 프랜차이즈단신버거킹 설날 와퍼 만원팩 출시 外
15 positive? 테스트 :  17 비교단어 :  출시 인덱스 :  15 버거킹 7일 한정판 와퍼커플팩 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 사진 버거킹 플라스틱 장난감 재활용 박스
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 더욱 풍성해진 언빌리버블이 4600원
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 11일부터 최대 300원 가격 인상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 제일기획 안대캠페인 이색 마케팅으로 버거킹 매출↑
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 정통 미국 남부의 맛 재현한 볼케이노칠리와퍼 볼케이노칠리X 무
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 불고기버거 단돈 1900원 4월1부터 딱 3일간 진행 눈길’
15 positive? 테스트 :  26 비교단어 :  출시 인덱스 :  15 버거킹 팬톤과 함께한 여름맞이 젤리에이드 3종 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 이럴 바에는 이벤트 안 했으면 좋겠어요 버거킹 이벤트 불고거 버거 수준
46 negativ

46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 아무때나 가도 할인혜택이
15 positive? 테스트 :  15 비교단어 :  출시 인덱스 :  15 버거킹 봄소풍 박스 킹박스 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 강남 지역 24시간 딜리버리 서비스 개시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 풍성한 고객 혜택 담은 5월 프로모션 진행
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 오늘20일부터 일주일간 버거킹 프리미엄 와퍼 단돈 3900원
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 RA인증 커피·핫초코 1000원 판매
27 positive? 테스트 :  19 비교단어 :  진출 인덱스 :  27 버거킹 와퍼 3000원 판매 한국 진출 30주년 기념
15 positive? 테스트 :  17 비교단어 :  출시 인덱스 :  15 버거킹 2월 시즌 한정 킹박스 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 봄맞이 와퍼팩’ 9900원 판매 프로모션
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 SAP ERP·구매조달솔루션 도입
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 이달 5일까지 와퍼와 콜라 5000원
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 인근 맥도날드 매장이 22년 만에 폐점하자 버거킹이 작별 인사를 전했다
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 콰트로치즈와퍼 5일간 3500원 판매
6 positive? 테스트 :  31 비교단어 :  수상 인덱스 :  6 버거킹 일자리 창출지수 우수 기업 고용노동부 장관 표창 수상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버

46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 프리미엄 와퍼 40 할인 판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 가격 올해 올려놓고 또 올려 도미노 인상 우려’
15 positive? 테스트 :  15 비교단어 :  출시 인덱스 :  15 버거킹 한정판 와퍼커플팩’ 출시
15 positive? 테스트 :  26 비교단어 :  출시 인덱스 :  15 버거킹 새우 풍미가 살아있는 쉬림프 치킨프라이 출시
11 positive? 테스트 :  13 비교단어 :  대박 인덱스 :  11 버거킹 와퍼주니어 11 대박
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 11일부터 8개 메뉴 100300원씩 가격 인상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 히어로 시즌3 3900원에 판매
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 수험생 응원 위해 프리미엄 와퍼 3종 3900원 할인
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼카 찍으면 와퍼세트가
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹2017 베스트 와퍼 위크’행사 진행
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 와퍼주니어 단 돈 1천900원”
15 positive? 테스트 :  15 비교단어 :  출시 인덱스 :  15 버거킹 한정판 몬스터 와퍼 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 꿈나무 마을 아이들과 함께하는 와퍼의 날
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 美 버거킹 음식 영수증에 쪼끄만 중국인’
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 올해의 크리에이티브 마케터 선정
15 positive? 테스트 

1 positive? 테스트 :  17 비교단어 :  제휴 인덱스 :  1 넥슨 야생의 땅 듀랑고×버거킹 제휴 프로모션 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 아로나민 버거킹·아이폰·구글 등과 동급 브랜드
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 워런버핏 버거킹 팀호튼스 인수 자금 지원
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 핫도그 단돈 천원 진짜 크고 맛있어’ 언제까지
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹의 젓가락’ 시식 광고 논란인종차별” 지적에 사과
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹와퍼 3종 3900원 할인 프로모션 어머 여긴 가야돼
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 베트남 리포트 亞식문화 비하 광고에 버거킹 보이콧 움직임
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 카리스마 장첸역 열연 윤계상 버거킹 새 얼굴 선정
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 2일부터 와퍼·불고기와퍼 등 일부 메뉴 100원 인상
15 positive? 테스트 :  20 비교단어 :  출시 인덱스 :  15 버거킹 더블킹 신메뉴 다이나믹BBQ 출시
15 positive? 테스트 :  20 비교단어 :  출시 인덱스 :  15 버거킹 더블킹 신메뉴 다이나믹BBQ 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹배달서비스 메뉴당 200원씩 가격 인상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 맛에 양을 더했다식품업계는 UP리지널 시대
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 Crew 채용의 날 진행
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 

이렇게 만든 데이터 프레임은

In [ ]:
def dftoCsv(my_title_df, num):
  my_title_df.to_csv(('./title_datas'+ str(num) +'.csv'), sep=',', na_rep='NaN', encoding='utf-8')

다음 코드를 활용하여 csv파일로 저장하였습니다.

 

학습데이터는 위의 코드를 통해 만들어진

버거킹기사 4,000개, 아오리라멘 기사 1,000개, 국대떡볶이기사 1,000개 데이터를 합쳐서 활용하였습니다.

train_dataset_1007.csv
0.44MB
테스트데이터는 맘스터치 1,500개의 기사데이터를 활용하였습니다.

test_dataset_1007.csv
0.11MB
 

 

2. 데이터 분석해보기
만들어진 csv파일을 google drive에 업로드하고 google colab에서 google drive를 마운트한 뒤 진행했습니다.

In [ ]:
import pandas as pd

train_data = pd.read_csv("./train_dataset_1007.csv")
test_data = pd.read_csv("./test_dataset_1007.csv")

train_data와 test_data를 pandas의 read_csv를 활용하여 dataframe으로 불러옵니다.

그 다음 matplotlib을 활용하여 -1, 0, 1 라벨별로 각각 몇개의 데이터가 존재하는지 확인해봅니다.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
train_data['label'].value_counts().plot(kind='bar')

In [ ]:
test_data['label'].value_counts().plot(kind='bar')

숫자로도 확인해봅니다. 1이 긍정 -1이 부정 0이 중립입니다.

In [ ]:
print(train_data.groupby('label').size().reset_index(name='count'))
print(test_data.groupby('label').size().reset_index(name='count'))

3. 모델을 만들기 위한 데이터 전처리 작업
먼저 각각의 제목을 토큰화 해주었습니다.

Okt형태소 분석기를 활용하였습니다.

In [ ]:
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

In [ ]:
import konlpy
from konlpy.tag import Okt
okt = Okt()
X_train = []
for sentence in train_data['title']:
  temp_X = []
  temp_X = okt.morphs(sentence, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_train.append(temp_X)

X_test = []
for sentence in test_data['title']:
  temp_X = []
  temp_X = okt.morphs(sentence, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_test.append(temp_X)

토큰화가 잘 되었는지 출력해보면 다음과 같습니다.

토큰화 한 단어를 컴퓨터가 인식할 수 있도록 정수인코딩을 해주었습니다.

In [ ]:
from keras.preprocessing.text import Tokenizer
max_words = 35000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

데이터의 최대길이 평균길이 그리고 길이를 기준으로 데이터의 분포가 어떠한지 확인해 보았습니다.

In [ ]:
print("제목의 최대 길이 : ", max(len(l) for l in X_train))
print("제목의 평균 길이 : ", sum(map(len, X_train))/ len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of Data')
plt.ylabel('number of Data')
plt.show()

In [ ]:
print("제목의 최대 길이 : ", max(len(l) for l in X_test))
print("제목의 평균 길이 : ", sum(map(len, X_test))/ len(X_test))
plt.hist([len(s) for s in X_test], bins=50)
plt.xlabel('length of Data')
plt.ylabel('number of Data')
plt.show()

값으로 들어갈 label -1, 0, 1을 컴퓨터가 보고 알수 있도록 one-hot encoding을 해주었습니다.

In [ ]:
import numpy as np

y_train = []
y_test = []



for i in range(len(train_data['label'])):
  if train_data['label'].iloc[i] == 1:
    y_train.append([0, 0, 1])
  elif train_data['label'].iloc[i] == 0:
    y_train.append([0, 1, 0])
  elif train_data['label'].iloc[i] == -1:
    y_train.append([1, 0, 0])
    
for i in range(len(test_data['label'])):
    if test_data['label'].iloc[i] == 1:
      y_test.append([0, 0, 1])
    elif test_data['label'].iloc[i] == 0:
      y_test.append([0, 1, 0])
    elif test_data['label'].iloc[i] == -1:
      y_test.append([1, 0, 0])
      
y_train = np.array(y_train)
y_test = np.array(y_test)

4. 모델 만들기

In [ ]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
max_len = 20 # 전체 데이터의 길이를 20로 맞춘다

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

먼저 필요한 것들을 import 해주고 pad_sequences를 활용하여 모든 데이터의 길이를 20으로 통일하였습니다.

In [ ]:
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.1)

긍정, 부정, 중립 3가지로 분류해야하니 LSTM, softmax, categorical_crossentropy를 사용하였습니다.

batch_size는 10 6,000개의 훈련데이터 중 10퍼센트인 600개는 validation_data로 활용하기위해 validation_split을 0.1을 부여하였습니다.

optimizer는 rmsprop을 사용하여 위와 같이 모델을 만들고 학습을 시켜보았습니다.

맘스터치관련 기사 제목 1,000개로 구성되어있는 테스트 데이터셋으로 평가해보니 94.27퍼센트가 나왔습니다.

생각보다 너무 잘나와서 조금 이상하지만 optimizer만 adam으로 바꿔 한번 더 해보았습니다.

이번엔 96.07%라는 결과가 나왔습니다.

In [ ]:
predict = model.predict(X_test)

In [ ]:
import numpy as np
predict_labels = np.argmax(predict, axis=1)
original_labels = np.argmax(y_test, axis=1)

In [ ]:
for i in range(30):
  print("기사제목 : ", test_data['title'].iloc[i], "/\t 원래 라벨 : ", original_labels[i], "/\t예측한 라벨 : ", predict_labels[i])